In [1]:
using GLPK
using JuMP

In [2]:
m = Model()
set_optimizer(m,GLPK.Optimizer);

In [3]:
mutable struct Instance
    n :: Int64
    D
    l
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        open(s) do f
            line = 1
            lineContent = readline(f)
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    D[line,col] = parse(Float64, lineSplited[col])
                end
                
                line += 1
            end
        end
        new(nNodes,D,l)
    end
end

In [4]:
I = Instance("instances/test.txt")

Instance(5, [0.0 2.0 … 3.0 1.0; 2.0 0.0 … 1.0 1.0; … ; 3.0 1.0 … 0.0 1.0; 1.0 1.0 … 1.0 0.0], 2)

In [5]:
# @variable(m, u[1:I.n]) # colocar real positivo # restrição remoção de subciclos
@variable(m, x[1:I.n,1:I.n,1:I.l], Bin)

5×5×2 Array{VariableRef, 3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  x[1,5,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]  x[2,5,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]  x[3,5,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]  x[4,5,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]  x[5,5,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  x[1,5,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]  x[2,5,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]  x[3,5,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]  x[4,5,2]
 x[5,1,2]  x[5,2,2]  x[5,3,2]  x[5,4,2]  x[5,5,2]

In [6]:
@constraint(m, [i=1:I.n], sum(x[i,j,1] for j=1:I.n)==1)
@constraint(m, [j=1:I.n], sum(x[i,j,1] for i=1:I.n)==1)

# @constraint(m, sum(x[i,j,1] for i=1:I.n, j=1:I.n)==I.n)

for l=2:I.l
#     @constraint(m, [i=1:I.n,j=1:I.n], x[i,j,l] <= sum((x[a,j,l-1]+x[i,a,l-1])/2 for a=1:I.n))
    @constraint(m, [i=1:I.n,j=1:I.n], x[i,j,l] >= sum(x[a,j,l-1]+x[i,a,1]-1 for a=1:I.n))
end

# dar exemplos
# caso caber na memoria, 

@constraint(m, [i=1:I.n], x[i,i,1] == 0)

5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1,1] == 0.0
 x[2,2,1] == 0.0
 x[3,3,1] == 0.0
 x[4,4,1] == 0.0
 x[5,5,1] == 0.0

In [ ]:
# @constraint(m, [i=1:I.n], y[1,i] == sum(x[i,j]*j for j=1:I.n)) # for lookahed=1

# for l=2:I.l
#     @constraint(m, [i=1:I.n], y[l,i] == y[l-1,y[l-1,i]])
# end

# @objective(m, Min, sum(I.D[i,y[s,i]] for s=1:I.l, i=1:I.n))

# restrição remoção de subciclos

# @objective(m, Min, sum(I.D[pi[i],pi[i+s]]*x[pi[i],pi[i+s]] for s=1:I.l, i=1:I.n))

# @objective(m, Min, sum(D[i,j]*x[i,j] for i=1:n, j=1:n))
# @constraint(m, Min, sum(D[i,j]*x[i,j] for i=1:n, j=1:n))

In [7]:
@objective(m, Min, sum(I.D[i,j] * x[i,j,s] for s=1:I.l, i=1:I.n, j=1:I.n))

2 x[1,2,1] + 2 x[1,3,1] + 3 x[1,4,1] + x[1,5,1] + 2 x[2,1,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] + 2 x[3,1,1] + x[3,2,1] + 2 x[3,4,1] + 3 x[3,5,1] + 3 x[4,1,1] + x[4,2,1] + 2 x[4,3,1] + x[4,5,1] + x[5,1,1] + x[5,2,1] + 3 x[5,3,1] + x[5,4,1] + 2 x[1,2,2] + 2 x[1,3,2] + 3 x[1,4,2] + x[1,5,2] + 2 x[2,1,2] + x[2,3,2] + x[2,4,2] + x[2,5,2] + 2 x[3,1,2] + x[3,2,2] + 2 x[3,4,2] + 3 x[3,5,2] + 3 x[4,1,2] + x[4,2,2] + 2 x[4,3,2] + x[4,5,2] + x[5,1,2] + x[5,2,2] + 3 x[5,3,2] + x[5,4,2]

In [8]:
optimize!(m)
objective_value(m)

6.0

In [9]:
println(m)

Min 2 x[1,2,1] + 2 x[1,3,1] + 3 x[1,4,1] + x[1,5,1] + 2 x[2,1,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] + 2 x[3,1,1] + x[3,2,1] + 2 x[3,4,1] + 3 x[3,5,1] + 3 x[4,1,1] + x[4,2,1] + 2 x[4,3,1] + x[4,5,1] + x[5,1,1] + x[5,2,1] + 3 x[5,3,1] + x[5,4,1] + 2 x[1,2,2] + 2 x[1,3,2] + 3 x[1,4,2] + x[1,5,2] + 2 x[2,1,2] + x[2,3,2] + x[2,4,2] + x[2,5,2] + 2 x[3,1,2] + x[3,2,2] + 2 x[3,4,2] + 3 x[3,5,2] + 3 x[4,1,2] + x[4,2,2] + 2 x[4,3,2] + x[4,5,2] + x[5,1,2] + x[5,2,2] + 3 x[5,3,2] + x[5,4,2]
Subject to
 x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] == 1.0
 x[2,1,1] + x[2,2,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] == 1.0
 x[3,1,1] + x[3,2,1] + x[3,3,1] + x[3,4,1] + x[3,5,1] == 1.0
 x[4,1,1] + x[4,2,1] + x[4,3,1] + x[4,4,1] + x[4,5,1] == 1.0
 x[5,1,1] + x[5,2,1] + x[5,3,1] + x[5,4,1] + x[5,5,1] == 1.0
 x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] == 1.0
 x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] == 1.0
 x[1,3,1] + x[2,3,1] + x[3,3,1] + x[4,3,1] + x[5,3,1] == 1.0
 x[1,4,1] + x[2,4,1

In [13]:
for i=1:n print(value(x[i]), " ") end

1.0 0.0 1.0 1.0 